# Load dataset to database

In this notebook, we are loading the contents of this kaggle Books Dataset into our chromaDB database, so the agent has some ground knowledge about a wide range of books.

[Books Dataset](https://www.kaggle.com/datasets/elvinrustam/books-dataset/)  

**Overview:**  
This dataset comprises information scraped from wonderbk.com, a popular online bookstore. The dataset contains details of 103,063 books, with key attributes such as title, authors, description, category, publisher, starting price, and publish date.

**Columns:**  
- Title: The title of the book.
- Authors: The authors of the book.
- Description: A brief description of the book.
- Category: The category or genre to which the book belongs.
- Publisher: The publishing house responsible for the book.
- Publish Date: The month in which the book was published.
- Price: The initial price of the book.

## Load and clean dataset

In [ ]:
import pandas as pd

books_df = pd.read_csv("datasets/BooksDataset.csv")
print(books_df.info())
books_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103082 entries, 0 to 103081
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Title         103082 non-null  object
 1   Authors       103082 non-null  object
 2   Description   70213 non-null   object
 3   Category      76912 non-null   object
 4   Publisher     103074 non-null  object
 5   Publish Date  103082 non-null  object
 6   Price         103082 non-null  object
dtypes: object(7)
memory usage: 5.5+ MB
None


,Title,Authors,Description,Category,Publisher,Publish Date,Price
0,Goat Brothers,"By Colton, Larry",NaN,"History , General",Doubleday,"Friday, January 1, 1993",Price Starting at $8.79
1,The Missing Person,"By Grumbach, Doris",NaN,"Fiction , General",Putnam Pub Group,"Sunday, March 1, 1981",Price Starting at $4.99
2,Don't Eat Your Heart Out Cookbook,"By Piscatella, Joseph C.",NaN,"Cooking , Reference",Workman Pub Co,"Thursday, September 1, 1983",Price Starting at $4.99
3,When Your Corporate Umbrella Begins to Leak: A...,"By Davis, Paul D.",NaN,NaN,Natl Pr Books,"Monday, April 1, 1991",Price Starting at $4.99
4,Amy Spangler's Breastfeeding : A Parent's Guide,"By Spangler, Amy",NaN,NaN,Amy Spangler,"Saturday, February 1, 1997",Price Starting at $5.32


In [39]:
# get rid of columns that are not needed
interesting_columns = ['Title', 'Authors', 'Description', 'Category', 'Publish Date']
books_df = books_df[interesting_columns]

In [40]:
# check for nan values
books_df.isna().sum()

Title               0
Authors             0
Description     32869
Category        26170
Publish Date        0
dtype: int64

In [41]:
# let's get rid of books without a description that are not very helpful
# empty categories are not important
books_df = books_df.dropna(subset=["Description"])
books_df.isna().sum()

Title              0
Authors            0
Description        0
Category        4908
Publish Date       0
dtype: int64

In [43]:
# reset index
books_df = books_df.reset_index(drop=True)
print(books_df.info())
books_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70213 entries, 0 to 70212
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         70213 non-null  object
 1   Authors       70213 non-null  object
 2   Description   70213 non-null  object
 3   Category      65305 non-null  object
 4   Publish Date  70213 non-null  object
dtypes: object(5)
memory usage: 2.7+ MB
None


,Title,Authors,Description,Category,Publish Date
0,Journey Through Heartsongs,"By Stepanek, Mattie J. T.",Collects poems written by the eleven-year-old ...,"Poetry , General","Saturday, September 1, 2001"
1,In Search of Melancholy Baby,"By Aksyonov, Vassily, Heim, Michael Henry, and...",The Russian author offers an affectionate chro...,"Biography & Autobiography , General","Monday, June 1, 1987"
2,The Dieter's Guide to Weight Loss During Sex,"By Smith, Richard","A humor classic, this tongue-in-cheek diet pla...","Health & Fitness , Diet & Nutrition , Diets","Sunday, January 1, 1978"
3,Germs : Biological Weapons and America's Secre...,"By Miller, Judith, Engelberg, Stephen, and Bro...","Deadly germs sprayed in shopping malls, bomb-l...","Technology & Engineering , Military Science","Monday, October 1, 2001"
4,The Good Book: Reading the Bible with Mind and...,"By Gomes, Peter J.","""The Bible and the social and moral consequenc...","Religion , Biblical Biography , General","Friday, May 1, 1998"


In [ ]:
# save the cleaned data
books_df.to_csv("datasets/BooksDatasetClean.csv", index=False)

## Store to database

In [59]:
from sentence_transformers import SentenceTransformer
import chromadb
from datetime import datetime
import uuid

BATCH_SIZE = 500  # Adjust the batch size based on your system's capacity
batch_texts = []
batch_metadata = []
batch_embeddings = []
batch_ids = []
counter = 0
rows_len = books_df.shape[0]

# initialise the database
chroma_client = chromadb.PersistentClient(path="../chroma_db/")

# get collection
collection_name = 'book_info'
for collection in chroma_client.list_collections():
    if collection.name == collection_name:
        break
else:
    raise ValueError(f"Collection name {collection_name} not found.")

# get embedder
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# iterate through dataframe and add to database in batches
for _, row in books_df.iterrows():

    # Prepare text for vectorization
    vectorized_text = f"{row['Title']} - {row['Description']} - {row['Category']}"
    
    # Prepare metadata
    metadata = {
        "Title": row['Title'],
        "Authors": row['Authors'],
        "Category": row['Category'],
        "Publish Date": row['Publish Date'],
        "added_on": datetime.now().isoformat()
    }
    
    # Prepare embeddings
    embedding_text = embedding_model.encode(vectorized_text)
    
    # Append to the batch lists
    batch_ids.append(str(uuid.uuid4()))
    batch_texts.append(vectorized_text)
    batch_metadata.append(metadata)
    batch_embeddings.append(embedding_text)
    counter += 1

    # When batch is full or we're at the last row, insert batch into database
    if len(batch_texts) == BATCH_SIZE or counter == rows_len:

        collection.add(
            ids=batch_ids,
            documents=batch_texts,
            metadatas=batch_metadata,
            embeddings=batch_embeddings
        )

        # Clear the batch lists for the next batch
        batch_ids.clear()
        batch_texts.clear()
        batch_metadata.clear()
        batch_embeddings.clear()

    # Print progress
    print(f"Added {counter}/{rows_len} ({int(counter * 100 / rows_len)}%) books to the database", end="\r", flush=True)
